<a href="https://colab.research.google.com/github/neorocke/DataScience/blob/main/%5BData_Analysis%5DFraus_Detection_%EC%B5%9C%EC%A2%85%EC%88%98%EC%A0%95%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 금융 부도 예측 모델 구축 (최종 수정본)

이 노트북에서는 가상의 금융 데이터를 생성하고, OpenAI의 임베딩 모델을 활용하여 특징을 추출한 후, 부도 예측 모델을 구축합니다. 또한 RAG(Retrieval-Augmented Generation)를 이용하여 지식 베이스를 통합하는 방법을 보여줍니다.

## 목차
1. [필요한 라이브러리 불러오기](#1)
2. [데이터 생성](#2)
3. [LLM을 활용한 특징 추출](#3)
4. [RAG를 활용한 지식 베이스 통합](#4)
5. [데이터 분할](#5)
6. [모델 학습](#6)
7. [모델 평가](#7)
8. [신규 고객 부도 확률 예측](#8)


---
## 주요 단계
1. 데이터 생성: 가상의 고객 프로필, 거래 데이터, 이력 데이터, 고객 리뷰 데이터를
생성합니다. 이 데이터는 부도 예측 모델을 학습하는 데 사용됩니다.
2. LLM을 활용한 특징 추출: OpenAI의 GPT-3 모델을 사용하여 고객 리뷰와 거래 데이터에서 임베딩을 추출합니다. 임베딩은 텍스트 데이터를 숫자 벡터로 변환하여 모델이 이해할 수 있도록 합니다.
3. RAG를 활용한 지식 베이스 통합: 지식 베이스를 생성하고, 해당 임베딩을 생성한 후, FAISS를 이용하여 검색 인덱스를 구축합니다. 이를 통해 모델은 입력 데이터와 관련된 지식을 검색하여 예측 성능을 향상시킬 수 있습니다.
4. 데이터 분할: 학습 데이터와 테스트 데이터로 분할합니다. 학습 데이터는 모델을 학습하는 데 사용되고, 테스트 데이터는 모델의 성능을 평가하는 데 사용됩니다.
5. 모델 학습: 로지스틱 회귀 모델을 학습합니다. 로지스틱 회귀는 이진 분류 문제에 널리 사용되는 모델입니다.
6. 모델 평가: 테스트 데이터로 모델을 평가합니다. 분류 보고서를 통해 모델의 성능을 확인할 수 있습니다.
7. 신규 고객 부도 확률 예측: 새로운 고객의 정보를 입력하여 부도 확률을 예측합니다. 모델은 입력 데이터를 기반으로 고객의 부도 확률을 계산합니다.
8. 추가 모델 학습

** 주요 기능:

- OpenAI GPT-3: 텍스트 데이터에서 임베딩을 추출하는 데 사용됩니다.
- FAISS: 지식 베이스 검색을 위한 인덱스를 구축하는 데 사용됩니다.
- 로지스틱 회귀: 부도 예측 모델로 사용됩니다.
- RAG: 지식 베이스를 통합하여 예측 성능을 향상시키는 데 사용됩니다.

---

**주의사항:** 이 코드를 실행하기 전에 실제 OpenAI API 키를 설정해야 합니다.


In [ ]:
!pip -q install openai==0.28 faiss-cpu pandas numpy scikit-learn
!pip -q install -U sentence-transformers

## 1. 필요한 라이브러리 불러오기

먼저 필요한 라이브러리를 불러옵니다.

In [ ]:
# 라이브러리 임포트
import numpy as np
import pandas as pd
import openai
import faiss
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

## 2. 데이터 생성

가상의 고객 프로필, 거래 데이터, 이력 데이터, 고객 리뷰 데이터를 생성합니다.

In [ ]:
# OpenAI API 키 설정 (여기에 실제 API 키를 입력하세요)
openai.api_key = ''

In [ ]:
# 고객 수 설정
num_customers = 1000

# 난수 생성 시드 고정 (재현성을 위해)
np.random.seed(42)

# 가상의 고객 프로필 생성
customer_profile = pd.DataFrame({
    'customer_id': range(num_customers),
    'age': np.random.randint(18, 80, num_customers),  # 나이: 18세부터 80세 사이의 정수
    'occupation': np.random.choice(['직장인', '자영업자', '학생', '무직'], num_customers),  # 직업
    'income': np.random.normal(50000, 15000, num_customers),  # 소득: 평균 5만원, 표준편차 1.5만원의 정규분포
    'credit_score': np.random.randint(300, 850, num_customers),  # 신용 점수: 300~850 사이의 정수
})

# 거래 데이터 생성
transaction_data = pd.DataFrame({
    'customer_id': np.repeat(range(num_customers), 10),  # 각 고객당 10건의 거래
    'transaction_time': pd.date_range(start='2022-01-01', periods=10*num_customers, freq='T'),  # 거래 시간
    'transaction_amount': np.random.uniform(-5000, 5000, 10*num_customers),  # 거래 금액: -5000부터 5000 사이의 실수
    'transaction_type': np.random.choice(['입금', '출금', '구매', '이체'], 10*num_customers),  # 거래 유형
})

# 이력 데이터 생성
history_data = pd.DataFrame({
    'customer_id': range(num_customers),
    'loan_amount': np.random.normal(20000, 10000, num_customers),  # 대출 금액
    'num_of_loans': np.random.randint(0, 5, num_customers),  # 대출 건수
    'previous_defaults': np.random.randint(0, 3, num_customers),  # 이전 부도 건수
    'delinquency_records': np.random.randint(0, 5, num_customers),  # 연체 기록 수
})

# 고객 리뷰 데이터 생성
customer_reviews = pd.DataFrame({
    'customer_id': range(num_customers),
    'customer_review': np.random.choice([
        '서비스에 만족합니다.',
        '대출 상환이 어려울 것 같습니다.',
        '금융 지원이 필요합니다.',
        '거래에 문제가 발생했습니다.',
        '감사합니다.',
        '직장을 잃었습니다.',
        '소득이 감소했습니다.'
    ], num_customers),
})

# 고객 프로필, 이력 데이터, 리뷰 데이터를 병합
data = customer_profile.merge(history_data, on='customer_id').merge(customer_reviews, on='customer_id')

# 부도 여부 생성 (조건에 따라 부도 여부 결정)
data['default'] = np.where(
    (data['credit_score'] < 600) |  # 신용 점수가 600 미만
    (data['income'] < 30000) |  # 소득이 3만원 미만
    (data['previous_defaults'] > 0) |  # 이전 부도 기록이 있음
    (data['delinquency_records'] > 2) |  # 연체 기록이 2회 초과
    (data['customer_review'].str.contains('어려울 것 같습니다|문제가 발생했습니다|직장을 잃었습니다|소득이 감소했습니다')),  # 부정적인 리뷰
    1, 0  # 조건 만족 시 1(부도), 아니면 0
)

# 직업(occupation) 컬럼을 원-핫 인코딩
data = pd.get_dummies(data, columns=['occupation'], drop_first=True)

# 데이터 상위 5개 행 출력
print(data.head())

<ipython-input-4-3d6e95dcb601>:19: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  'transaction_time': pd.date_range(start='2022-01-01', periods=10*num_customers, freq='T'),  # 거래 시간


   customer_id  age        income  credit_score   loan_amount  num_of_loans  \
0            0   56  55362.893658           828  17143.914826             4   
1            1   69  41190.958734           482  14504.622014             3   
2            2   46  47108.303191           340  12899.409434             2   
3            3   32  47689.449260           696  22824.504893             2   
4            4   60  70352.851613           793   9899.135411             2   

   previous_defaults  delinquency_records  customer_review  default  \
0                  2                    1       직장을 잃었습니다.        1   
1                  1                    4           감사합니다.        1   
2                  2                    2    금융 지원이 필요합니다.        1   
3                  2                    0  거래에 문제가 발생했습니다.        1   
4                  0                    0      서비스에 만족합니다.        0   

   occupation_자영업자  occupation_직장인  occupation_학생  
0            False           False           T

## 3. LLM을 활용한 특징 추출

OpenAI의 GPT-3를 사용하여 고객 리뷰와 거래 데이터에서 임베딩을 추출합니다. 배치 처리를 사용하여 API 호출 횟수를 줄였습니다.

In [ ]:
# 임베딩 함수 정의 (배치 처리)
# 열의 개수는 임베딩 벡터의 차원에 따라 결정됩니다 (예: text-embedding-3-small 모델의 경우 벡터 1536개 반환).
def get_embeddings(text_list):
    # 텍스트 리스트를 받아 임베딩 리스트를 반환하는 함수
    response = openai.Embedding.create(
        input=text_list,
        model='text-embedding-3-small'  # 사용 모델
    )
    embeddings = [np.array(data_point['embedding'], dtype='float32') for data_point in response['data']]
    return embeddings

In [ ]:
# 고객 리뷰 임베딩 생성
review_texts = data['customer_review'].tolist()
review_embeddings = get_embeddings(review_texts)
data['review_embedding'] = review_embeddings

In [ ]:
# 거래 데이터의 텍스트 특징 추출
transaction_data['transaction_text'] = transaction_data['transaction_type'] + ' ' + transaction_data['transaction_amount'].astype(str)
transaction_texts = transaction_data.groupby('customer_id')['transaction_text'].apply(lambda x: ' '.join(x)).reset_index()

# 거래 텍스트 임베딩 생성
transaction_text_list = transaction_texts['transaction_text'].tolist()
transaction_embeddings = get_embeddings(transaction_text_list)
transaction_texts['transaction_embedding'] = transaction_embeddings

# 거래 임베딩을 고객 데이터에 병합
data = data.merge(transaction_texts[['customer_id', 'transaction_embedding']], on='customer_id', how='left')

In [ ]:
data.head(2)

,customer_id,age,income,credit_score,loan_amount,num_of_loans,previous_defaults,delinquency_records,customer_review,default,occupation_자영업자,occupation_직장인,occupation_학생,review_embedding,transaction_embedding
0,0,56,55362.893658,828,17143.914826,4,2,1,직장을 잃었습니다.,1,False,False,True,"[0.0053709284, 0.05951845, -0.029351564, -0.03...","[0.0048638415, -0.015428931, -0.034345146, 0.0..."
1,1,69,41190.958734,482,14504.622014,3,1,4,감사합니다.,1,False,True,False,"[-0.020092405, -0.05244388, -0.058053963, 0.00...","[0.020721996, -0.0061204466, -0.03261773, 0.01..."


In [ ]:
# 임베딩 벡터를 배열로 변환

review_embeddings = np.vstack(data['review_embedding'].values)
transaction_embeddings = np.vstack(data['transaction_embedding'].values)

In [ ]:
print(review_embeddings.shape)
print(transaction_embeddings.shape)

(1000, 1536)
(1000, 1536)


In [ ]:
# 숫자형 특징 선택
numeric_features = ['age', 'income', 'credit_score', 'loan_amount', 'num_of_loans', 'previous_defaults', 'delinquency_records']
numeric_data = data[numeric_features].values

# 범주형 변수 처리
occupation_features = [col for col in data.columns if col.startswith('occupation_')]
occupation_data = data[occupation_features].values

# 전체 특징 벡터 생성
X = np.hstack((numeric_data, occupation_data, review_embeddings, transaction_embeddings))
y = data['default'].values

## 4. RAG를 활용한 지식 베이스 통합

지식 베이스를 생성하고, 해당 임베딩을 생성한 후, FAISS를 이용하여 검색 인덱스를 구축합니다.

In [ ]:
# 지식 베이스 생성
knowledge_base = pd.DataFrame({
    'text': [
        '신용 점수가 낮으면 부도 확률이 높습니다.',
        '이전 부도 기록이 있으면 부도 위험이 증가합니다.',
        '연체 기록이 많으면 부도 가능성이 높습니다.',
        '소득이 낮거나 불안정하면 부도 위험이 증가합니다.',
        '고객의 부정적인 리뷰는 부도 위험의 신호입니다.',
    ]
})

# 지식 베이스 임베딩 생성
kb_texts = knowledge_base['text'].tolist()
kb_embeddings = get_embeddings(kb_texts)
knowledge_base['embedding'] = kb_embeddings
kb_embeddings = np.vstack(kb_embeddings)
print(kb_embeddings.shape)

(5, 1536)


In [ ]:
# FAISS를 이용한 검색 인덱스 생성
dimension = kb_embeddings.shape[1] # 지식 베이스 텍스트 임베딩 벡터 1536
index = faiss.IndexFlatL2(dimension)
index.add(kb_embeddings)

# 지식 검색 함수 정의
# index.search() 함수는 FAISS 인덱스에서 쿼리 벡터와 가장 유사한 벡터를 검색하는 함수입니다.
# np.array([query_embedding]): 쿼리 임베딩을 NumPy 배열로 변환합니다. FAISS는 NumPy 배열을 입력으로 받습니다.
# k: 검색할 가장 유사한 벡터의 개수입니다.
# D: 쿼리 벡터와 검색된 벡터 사이의 거리를 담고 있는 NumPy 배열입니다.
# I: 검색된 벡터의 인덱스를 담고 있는 NumPy 배열입니다.
def retrieve_knowledge(query_embedding, k=1):
    D, I = index.search(np.array([query_embedding]), k)
    return knowledge_base.iloc[I[0]]['text'].values

 ## 4.1 고객의 리뷰와 거래 내역 정보를 결합하여 임베딩 벡터를 생성
- **리뷰 임베딩**: 고객 리뷰 텍스트에 담긴 정보를 벡터 형태로 표현합니다. 고객의 감정, 의견, 경험 등을 반영할 수 있습니다.
- **거래 임베딩**: 고객의 거래 내역에 담긴 정보를 벡터 형태로 표현합니다. 거래 횟수, 금액, 유형 등을 반영할 수 있습니다.
이 두 가지 임베딩을 단순히 연결 (concatenate) 하는 대신 요소별 덧셈을 하는 이유는 각 정보의 중요도를 동등하게 반영하면서 두 정보를 융합하기 위해서입니다.


요소별 덧셈
- 정보 융합: 두 벡터의 정보를 하나의 벡터에 압축합니다.
- 상호 작용 반영: 리뷰와 거래 정보 간의 상호 작용을 어느 정도 반영할 수 있습니다. 예를 들어, 긍정적인 리뷰와 큰 금액의 거래는 긍정적인 방향으로 벡터 값을 강화할 수 있습니다.

In [ ]:
# 리뷰 임베딩과 거래 임베딩 요소별 덧셈
combined_embeddings = review_embeddings + transaction_embeddings

# 관련 지식을 저장할 컬럼 추가
data['related_knowledge'] = ''

# 각 데이터에 대해 관련 지식 검색
for i in range(len(combined_embeddings)):
    combined_embedding = combined_embeddings[i]
    related_knowledge = retrieve_knowledge(combined_embedding)
    data.at[i, 'related_knowledge'] = ' '.join(related_knowledge)

data[:2]

,customer_id,age,income,credit_score,loan_amount,num_of_loans,previous_defaults,delinquency_records,customer_review,default,occupation_자영업자,occupation_직장인,occupation_학생,review_embedding,transaction_embedding,related_knowledge
0,0,56,55362.893658,828,17143.914826,4,2,1,직장을 잃었습니다.,1,False,False,True,"[0.0053709284, 0.05951845, -0.029351564, -0.03...","[0.0048638415, -0.015428931, -0.034345146, 0.0...",연체 기록이 많으면 부도 가능성이 높습니다.
1,1,69,41190.958734,482,14504.622014,3,1,4,감사합니다.,1,False,True,False,"[-0.020092405, -0.05244388, -0.058053963, 0.00...","[0.020721996, -0.0061204466, -0.03261773, 0.01...",연체 기록이 많으면 부도 가능성이 높습니다.


In [ ]:
# 관련 지식 임베딩 생성
related_knowledge_texts = data['related_knowledge'].tolist()
knowledge_embeddings = get_embeddings(related_knowledge_texts)

# 향상된 입력 데이터 생성
enhanced_embeddings = []
for i in range(len(X)):
    enhanced_embedding = np.hstack((X[i], knowledge_embeddings[i]))
    enhanced_embeddings.append(enhanced_embedding)

X_enhanced = np.array(enhanced_embeddings)

In [ ]:
X_enhanced.shape

(1000, 4618)

## 5. 데이터 분할

학습 데이터와 테스트 데이터로 분할합니다.

In [ ]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_enhanced, y, test_size=0.2, random_state=42)

## 6. 모델 학습

로지스틱 회귀 모델을 학습합니다.

In [ ]:
# 모델 학습
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000)

## 7. 모델 평가

테스트 데이터로 모델을 평가합니다.

In [ ]:
# 모델 평가
y_pred = model.predict(X_test)
print("\n분류 보고서:")
print(classification_report(y_test, y_pred))


분류 보고서:
              precision    recall  f1-score   support

           0       0.67      0.80      0.73         5
           1       0.99      0.99      0.99       195

    accuracy                           0.98       200
   macro avg       0.83      0.89      0.86       200
weighted avg       0.99      0.98      0.99       200



## 8. 신규 고객 부도 확률 예측

새로운 고객의 정보를 입력하여 부도 확률을 예측합니다.

In [ ]:
# 신규 고객 정보
new_customer = {
    'age': 45,
    'occupation': '무직',
    'income': 25000,
    'credit_score': 580,
    'loan_amount': 30000,
    'num_of_loans': 2,
    'previous_defaults': 1,
    'delinquency_records': 3,
    'customer_review': '최근에 직장을 잃었고, 상환이 어려울 것 같습니다.',
    'transaction_text': '출금 -3000 입금 2000 구매 -5000 출금 -2000'
}

# 범주형 변수 처리
occupation_encoded = [0, 0, 0]  # ['occupation_무직', 'occupation_자영업자', 'occupation_직장인']
if new_customer['occupation'] == '무직':
    occupation_encoded[0] = 1
elif new_customer['occupation'] == '자영업자':
    occupation_encoded[1] = 1
elif new_customer['occupation'] == '직장인':
    occupation_encoded[2] = 1

# 임베딩 생성
new_texts = [new_customer['customer_review'], new_customer['transaction_text']]
new_embeddings = get_embeddings(new_texts)
new_review_embedding = new_embeddings[0]
new_transaction_embedding = new_embeddings[1]

# 관련 지식 검색 및 임베딩 결합
combined_embedding = new_review_embedding + new_transaction_embedding
related_knowledge = retrieve_knowledge(combined_embedding)
knowledge_embedding = get_embeddings([' '.join(related_knowledge)])[0]

# 특징 벡터 생성
numeric_features = [new_customer['age'], new_customer['income'], new_customer['credit_score'],
                    new_customer['loan_amount'], new_customer['num_of_loans'], new_customer['previous_defaults'],
                    new_customer['delinquency_records']]

new_feature = np.hstack((
    numeric_features,
    occupation_encoded,
    new_review_embedding,
    new_transaction_embedding,
    knowledge_embedding
))

# 부도 확률 예측
default_prob = model.predict_proba([new_feature])[0][1]
print(f"\n신규 고객의 부도 확률: {default_prob * 100:.2f}%")


신규 고객의 부도 확률: 99.96%


## 9. 베스트 모델 추가 분석

- LightGBM: 효율적이고 빠른 그라디언트 부스팅 알고리즘.
- XGBoost: 성능이 매우 뛰어난 부스팅 알고리즘.
- CatBoost: 범주형 데이터에 최적화된 부스팅 알고리즘.
- Random Forest: 대표적인 트리 기반 앙상블 모델.
- Logistic Regression: 비교적 단순한 선형 모델.


In [ ]:
!pip -q install lightgbm xgboost catboost

In [ ]:
import warnings
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import classification_report

# 경고 메시지 무시
warnings.filterwarnings("ignore")

# 모델 리스트
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(),
    "LightGBM": LGBMClassifier(max_depth=10, n_estimators=100, min_child_samples=20, learning_rate=0.1, verbose=-1),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    # "CatBoost": CatBoostClassifier(verbose=0)
}

# 교차 검증을 통한 성능 평가 함수
def evaluate_models(models, X_train, y_train):
    best_model = None
    best_score = 0
    for name, model in models.items():
        scores = cross_val_score(model, X_train, y_train, cv=5, scoring='f1')
        mean_score = scores.mean()
        print(f"{name} f1: {mean_score:.4f}")

        if mean_score > best_score:
            best_score = mean_score
            best_model = model
    print(f"\n가장 성능이 좋은 모델: {best_model.__class__.__name__}, 정확도: {best_score:.4f}")
    return best_model

# 최적 모델 선택 및 학습
best_model = evaluate_models(models, X_train, y_train)
best_model.fit(X_train, y_train)


Logistic Regression f1: 0.9869
Random Forest f1: 0.9717
LightGBM f1: 0.9967
XGBoost f1: 0.9960

가장 성능이 좋은 모델: LGBMClassifier, 정확도: 0.9967


LGBMClassifier(max_depth=10, verbose=-1)

In [ ]:

# 부도 확률 예측
default_prob = best_model.predict_proba([new_feature])[0][1]
print(f"\n신규 고객의 부도 확률: {default_prob * 100:.2f}%")



신규 고객의 부도 확률: 100.00%
